# Hassediagram-Editor

- You will find some example data files and generate Hassediagrams.
- Parameters like colors and selected nodes can be set as you like.

### Configuration for D3

In [1]:
from IPython.core.display import HTML
import hd3d_lib
HTML('<script src="./lib/d3/d3.min.js"></script>')

### Import modules

In [2]:
import pathlib
from pyhasse.core.csv_io import CSVReader
from pyhasse.core.matrix import Matrix
from pyhasse.core.order import Order
from pyhasse.core.hddata import HDData
import json
import random
from IPython.core.display import display, HTML
import random
#random.seed(42)
from string import Template


### Import data and preparatory work

Put your own data sets into the folder csvdata.

In [3]:
TESTFILENAME = '/csvdata/zeta-test.csv'
#TESTFILENAME = '/csvdata/acm-houses.csv'
#TESTFILENAME = '/csvdata/chain-demo.csv'
#TESTFILENAME = '/csvdata/chain-pollution.csv'
#TESTFILENAME = '/csvdata/copeland_demo.csv'
#TESTFILENAME = '/csvdata/fuzzy-demo.csv'
#TESTFILENAME = '/csvdata/main19chemicals.csv'
#TESTFILENAME = '/csvdata/pesticides-demo.txt'
#TESTFILENAME = '/csvdata/spyout-demo.csv'
#TESTFILENAME = '/csvdata/zeta-test.csv'
#TESTFILENAME = '/csvdata/check-K.csv'
#TESTFILENAME = '/csvdata/kassel2.txt'

HERE = pathlib.Path('__file__').parent
csv = CSVReader(fn=str(HERE) + TESTFILENAME, ndec=3)
# prepare dataset (reduced system)
matrix = Matrix(csv.data, csv.obj, csv.prop, reduced=True)

### Calculate data and configure the Hassediagram

In [4]:
hd = HDData(matrix, csv)
data_dict = hd.jsondata()
data_dict['dmred'] = matrix.data
selected_obj = ''
data_dict['lst_downsets'] = []
data_dict['lst_upsets'] = []
data_dict['lst_incomparables'] = []
    
settings = {
    "preselected": [],
    "arrowShow": 1,
    "unselectedHide": 0,
    "edgeHighlight": 1,
    "arrowWidth": 20,
    "arrowSideRight": 1,
    "unselectedNodeColor": "#00ff00",
    "selectedNodeColor": "#ffbfff",
    "arrowColor": "#ff0000",
    "textColor": "#000000",
    "lineColor": "#FFF",
    "selectedBkgColor": "#008",
    "bkgGradient1": "#2db4ff",
    "bkgGradient2": "#2db4ff",
    "shortLength": 4,
    "hNodeDist": 2,
    "vNodeDist": 2,
    "r": 20,         # node size
    "bkgR": 20 + 5,  # background upset/downset
    "frameHeight": 600
}

data_dict.update(settings)

######################################
[[0, 3, 4], [1], [2], [5], [6]]
{'mx_cover': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [1, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0]], 'mx_zeta': [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0], [1, 1, 1, 0, 1]], 'mx_obj_red': [[1.0, 2.0], [2.0, 1.0], [3.0, 3.0], [4.0, 4.0], [3.0, 5.0]], 'lst_obj_red': ['a', 'b', 'c', 'f', 'g'], 'int_levels': 3, 'fd_levels': [[3, 4], [2], [0, 1]], 'int_max_objs': 2}
########################################
[0, 3, 4]
[1]
[2]
[5]
[6]


In [5]:
precision = 4
order = Order(csv.data,
              csv.rows,
              csv.cols)
zeta =order.calc_relatmatrix(csv.data, csv.rows, csv.cols)
zeta = order.calc_relatmatrix(
    datamatrix=matrix.data,
    rows=matrix.rows,
    cols=matrix.cols,
    prec=precision)

data_dict['lst_downsets'] = order.calc_downset(zeta, matrix.rows)
data_dict['lst_upsets'] = order.calc_upset(zeta, matrix.rows)


### For navigation by upset, downset or both, select the generating node

In [6]:
#node = 'c'
node = ''
data_dict['preselected'] = []

if node != '':
    node_index = data_dict['lst_obj_red'].index(node)
    # Upsets
    upsets = [data_dict['lst_obj_red'][i] for i in data_dict['lst_upsets'][node_index]]
    # Downset
    downsets = [data_dict['lst_obj_red'][i] for i in data_dict['lst_downsets'][node_index]]

### select by commenting or commenting out whether or not  a downset, an upset or both is wanted.

In [7]:
if node != '':
    data_dict['preselected'] = upsets + downsets
    #data_dict['preselected'] = upsets
    #data_dict['preselected'] = downsets

In [8]:
#eqm = csv.generate_eqcl(csv.obj,csv.eqm)

eqm, rred =  matrix.generate_equivalenceclasses(csv.data, csv.rows, csv.cols, prec=None)
print(f'{"Pointer":>8} {"object label represents":^30} {"equivalence class":^20}')
print(f'{59 * "-"}')
for idx, value in enumerate(matrix.obj):
      print(f'{csv.obj.index(value):>8} {value:^30} {str(eqm[idx]):^20}')     

 Pointer    object label represents      equivalence class  
-----------------------------------------------------------
       0               a                     [0, 3, 4]      
       1               b                        [1]         
       2               c                        [2]         
       5               f                        [5]         
       6               g                        [6]         


# Draw Hassediagram
### Navigation
- zoom (smooth) -- *Mouse-Wheel*
- zoom to node -- *Right-Click* on node
- move the diagram -- *Click+Drag*
- move a node -- *Click on node+Drag*
- select one node -- *Click*
- toggle a node's selection or select multiple nodes -- *Shift+Click*


In [9]:
HTML(hd3d_lib.draw_hasse('diagramd3_editor', data_dict))